# Naive Bayes

Der "Naive Bayes Classifier" ist ein *Klassifizierungs*-Algorithmus, der auf dem *Satz von Bayes* aufbaut.

## Fragen

- Was ist Maschinenlernen?
- Wie funktioniert Naive Bayes?
- Wann kann ich Naive Bayes benutzen?

# Was ist Maschinenlernen?